In [2]:
from confluent_kafka import Consumer, KafkaError
from influxdb import InfluxDBClient
import time 
import json
import datetime
import random


c = Consumer({
    'bootstrap.servers': 'localhost:9092',
    'group.id': 'mygroup',
    'auto.offset.reset': 'earliest'
})
user = 'root'
password = 'root'
dbname = 'vehicle'
json_body=[]
data={}
data["measurement"] = "vehicle_signals"
tag={}
tag["vachicle_id"]= "ec0e7d95-562e-4de3-b3f6-8f2cd6113d22"
tag["location"]="Aachen"
data["tags"]=tag
client = InfluxDBClient('localhost', 8086, user, password, dbname)

c.subscribe(['vehicle_signals'])

while True:
    try:
        msg = c.poll(1.0)

    except SerializerError as e:
        print("Message deserialization failed for {}: {}".format(msg, e))
        break

    if msg is None:
        continue

    if msg.error():
        print("AvroConsumer error: {}".format(msg.error()))
        continue

    print(msg.value().decode('utf-8'))
    json_body=[]
    data["time"]= datetime.datetime.utcnow()
    data["fields"]= json.loads(msg.value().decode('utf-8'))
    json_body.append(data)
    print(json_body)
    try :
        client.write_points(json_body)
    except :
        print("Error")
     

c.close()

AvroConsumer error: KafkaError{code=_PARTITION_EOF,val=-191,str="Broker: No more messages"}
{"battery_charging": "yes", "battery_charging_current": 16, "battery_health": 79, "battery_loading_capacity": 11, "battery_state_of_charge": 100, "battery_total_kwh_capacity": 17.5, "brake_fluid_level": 100, "calculated_remaining_distance": 150, "central_locking_system": "closed", "distance_to_object_back": 14.866637640919166, "distance_to_object_bottom": 1.0145246953549614, "distance_to_object_front": 16.514227251193883, "distance_to_object_left": 7.817024611912153, "distance_to_object_right": 1.3421563147965077, "distance_trip": 0, "door_disc_front_left": "open", "door_disc_front_right": "open", "door_front_left": "open", "door_front_right": "open", "drive_mode": "sport", "flash": "on", "heated_seats": "on", "high_beam": "on", "infotainment": "on", "infotainment_volume": 5, "location": "51,7", "mileage": 9872, "motor_control_lamp": "on", "person_count": 3, "pulse_sensor_steering_wheel": 73, "p

{"battery_charging": "yes", "battery_charging_current": 16, "battery_health": 55, "battery_loading_capacity": 11, "battery_state_of_charge": 100, "battery_total_kwh_capacity": 17.5, "brake_fluid_level": 100, "calculated_remaining_distance": 150, "central_locking_system": "closed", "distance_to_object_back": 3.055372494994937, "distance_to_object_bottom": 1.5405108586405047, "distance_to_object_front": 18.675044261380485, "distance_to_object_left": 17.199114962739202, "distance_to_object_right": 17.00981269248469, "distance_trip": 0, "door_disc_front_left": "open", "door_disc_front_right": "open", "door_front_left": "open", "door_front_right": "open", "drive_mode": "sport", "flash": "on", "heated_seats": "on", "high_beam": "on", "infotainment": "on", "infotainment_volume": 5, "location": "51,7", "mileage": 5793, "motor_control_lamp": "on", "person_count": 3, "pulse_sensor_steering_wheel": 73, "power_consumption": 0, "rain_sensor": "no_rain", "rear_running_lights": "on", "side_lights": "

{"battery_charging": "yes", "battery_charging_current": 16, "battery_health": 21, "battery_loading_capacity": 11, "battery_state_of_charge": 100, "battery_total_kwh_capacity": 17.5, "brake_fluid_level": 100, "calculated_remaining_distance": 150, "central_locking_system": "closed", "distance_to_object_back": 6.579780035880482, "distance_to_object_bottom": 7.8010645296053465, "distance_to_object_front": 8.83602766618427, "distance_to_object_left": 16.594990542317714, "distance_to_object_right": 10.18991454546647, "distance_trip": 0, "door_disc_front_left": "open", "door_disc_front_right": "open", "door_front_left": "open", "door_front_right": "open", "drive_mode": "sport", "flash": "on", "heated_seats": "on", "high_beam": "on", "infotainment": "on", "infotainment_volume": 5, "location": "51,7", "mileage": 199, "motor_control_lamp": "on", "person_count": 3, "pulse_sensor_steering_wheel": 73, "power_consumption": 0, "rain_sensor": "rain", "rear_running_lights": "on", "side_lights": "on", "

NameError: name 'SerializerError' is not defined